Эта тетрадка содержит примеры и упражнения по двум основным методам векторизации текста: Bag of Words (мешок слов) и TF-IDF (Term Frequency-Inverse Document Frequency).

In [ ]:
import re
import pandas as pd

**Часть 1: Исходные данные**

In [2]:
# Наш корпус документов
documents = [
    "Машинное обучение - это подраздел искусственного интеллекта.",
    "Нейронные сети широко используются в машинном обучении.",
    "Глубокое обучение основано на многослойных нейронных сетях.",
    "Искусственный интеллект имитирует когнитивные функции человека.",
    "Компьютерное зрение и обработка языка - примеры применения ИИ."
]

# Вывод документов с номерами
for i, doc in enumerate(documents, 1):
    print(f"Документ {i}: {doc}")

Документ 1: Машинное обучение - это подраздел искусственного интеллекта.
Документ 2: Нейронные сети широко используются в машинном обучении.
Документ 3: Глубокое обучение основано на многослойных нейронных сетях.
Документ 4: Искусственный интеллект имитирует когнитивные функции человека.
Документ 5: Компьютерное зрение и обработка языка - примеры применения ИИ.


**Часть 2: Предобработка**

In [ ]:
for i in range(len(documents)):
    # Приведение к нижнему регистру
    documents[i] = documents[i].lower()
    # Удаление пунктуации
    documents[i] = re.sub(r'[^\w\s]', '', documents[i])

  # Вывод предобработанных документов
print("Предобработанные документы:")
for i, doc in enumerate(documents, 1):
    print(f"Документ {i}: {doc}")

**Часть 3: Модель Bag of Words (мешок слов)**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  # Для создания Bag of Words

3.1 Создание матрицы Bag of Words

In [3]:
# Создание векторизатора
count_vectorizer = CountVectorizer()

# Обучение векторизатора и преобразование документов
bow_matrix = count_vectorizer.fit_transform(documents)

# Получение списка фичей (слов)
feature_names = count_vectorizer.get_feature_names_out()

# Преобразование разреженной матрицы в плотную для наглядности
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=feature_names,
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы Bag of Words
print("Матрица Bag of Words:")
print(bow_df)

NameError: name 'CountVectorizer' is not defined

Получение списка фичей (слов)

feature_names = count_vectorizer.get_feature_names_out()
Что происходит:
Этот шаг извлекает из обученного CountVectorizer все уникальные слова (или n-граммы), которые были использованы при создании матрицы Bag of Words.

Зачем это нужно:
* Интерпретация результатов: Без этого шага у нас была бы только числовая матрица, но мы не знали бы, какой столбец соответствует какому слову.
* Понимание словаря: Метод get_feature_names_out() возвращает список всех уникальных терминов, которые векторизатор извлек из документов и включил в свой словарь.
* Подготовка к визуализации: Имена признаков необходимы для создания понятной таблицы с метками строк и столбцов.

**Часть 4: Модель TF-IDF**

4.1 Создание матрицы TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# Создание TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer()

# Обучение векторизатора и преобразование документов
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы TF-IDF
print("Матрица TF-IDF:")
print(tfidf_df)

NameError: name 'TfidfVectorizer' is not defined

**Часть 5. Сравнение результатов**

In [5]:
import numpy as np

In [6]:
# Суммируем значения по всем документам для BOW
# Для BOW просто суммируем частоты слов по всем документам
bow_sum = np.sum(bow_matrix.toarray(), axis=0)

# Создаем словарь слово -> суммарная частота
word_bow_dict = dict(zip(feature_names, bow_sum))

# Находим топ-3 слова по BOW (самые частотные в коллекции)
top_bow_words = sorted(word_bow_dict.items(), key=lambda x: x[1], reverse=True)[:3]

print("=== Топ-3 слова по всей коллекции ===\n")
print("Топ-3 слова по BOW (самые частые в коллекции):")
for word, count in top_bow_words:
    print(f"- {word}: {count} раз")

# Для TF-IDF нужно суммировать значения по всем документам
tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)

# Создаем словарь слово -> суммарный TF-IDF вес
word_tfidf_dict = dict(zip(feature_names, tfidf_sum))

# Находим топ-3 слова по TF-IDF (с наибольшим суммарным весом по всей коллекции)
top_tfidf_words = sorted(word_tfidf_dict.items(), key=lambda x: x[1], reverse=True)[:3]

print("\nТоп-3 слова по TF-IDF (с наибольшим весом по всей коллекции):")
for word, score in top_tfidf_words:
    print(f"- {word}: {score:.4f}")

# Показываем разницу между списками
print("\nСравнение топ-3 слов:")
bow_words = [word for word, _ in top_bow_words]
tfidf_words = [word for word, _ in top_tfidf_words]

common_words = set(bow_words) & set(tfidf_words)
bow_only = set(bow_words) - common_words
tfidf_only = set(tfidf_words) - common_words

if common_words:
    print(f"Общие слова в обоих топ-3: {', '.join(common_words)}")
if bow_only:
    print(f"Только в BOW топ-3: {', '.join(bow_only)}")
if tfidf_only:
    print(f"Только в TF-IDF топ-3: {', '.join(tfidf_only)}")

NameError: name 'bow_matrix' is not defined

**Самостоятельная работа**

ЗАДАНИЕ:
1. Предобработайте эти документы (удалите стоп-слова, приведите к нижнему регистру)
2. Создайте матрицу Bag of Words
3. Создайте матрицу TF-IDF
4. Найдите и выведите топ-3 важных слова для каждого документа по Bag of Words
5. Найдите и выведите топ-3 важных слова для каждого документа по TF-IDF
6. Проанализируйте разницу между результатами и объясните её


In [ ]:
# Создаем коллекцию документов
exercise_documents = [
    "Япония поражает сочетанием древних традиций и футуристических технологий. Токио с его небоскребами и неоновыми огнями соседствует с тихими храмами и садами. Весной цветение сакуры превращает страну в розовое облако. Японская кухня, от суши до рамена, предлагает уникальные вкусовые ощущения. Синкансэны позволяют комфортно перемещаться между городами.",

    "Исландия — страна потрясающих природных контрастов. Ледники соседствуют с действующими вулканами, а горячие гейзеры бьют среди снежных равнин. Северное сияние зимой и незаходящее солнце летом создают ощущение другой планеты. Голубая лагуна с ее геотермальными водами — идеальное место для расслабления после долгих пеших походов по национальным паркам.",

    "Таиланд привлекает путешественников белоснежными пляжами и кристально чистой водой. Острова Пхукет и Самуи предлагают роскошные курорты и активный отдых. Бангкок поражает контрастами: золотые храмы соседствуют с оживленными рынками и современными торговыми центрами. Тайская кухня с ее острыми ароматами и экзотическими фруктами — отдельное гастрономическое путешествие.",

    "Италия — настоящий музей под открытым небом. Рим хранит наследие древней империи в Колизее и Форуме. Венеция очаровывает каналами и гондолами. Флоренция — сокровищница искусства эпохи Возрождения. Побережье Амальфи и озеро Комо предлагают живописные пейзажи. Итальянская кухня, от пасты до джелато, заслуженно считается одной из лучших в мире.",

    "Перу хранит тайны древних цивилизаций. Мачу-Пикчу, затерянный город инков, привлекает туристов со всего мира. Линии Наска, гигантские рисунки на плато, до сих пор остаются загадкой. Озеро Титикака поражает своими плавучими островами, на которых живут местные племена. Перуанская кухня, с ее свежими морепродуктами и разнообразием картофеля, переживает всемирное признание.",

    "Марокко — это калейдоскоп красок и ароматов. Медины Феса и Марракеша с их узкими улочками и шумными базарами погружают в атмосферу арабских сказок. Пустыня Сахара предлагает незабываемые ночи под звездами в берберских лагерях. Атласские горы привлекают любителей трекинга. Марокканская кухня славится тажинами и кус-кусом, приправленными экзотическими специями.",

    "Новая Зеландия — рай для любителей природы и активного отдыха. Фьорды Милфорд-Саунд, ледники Южных Альп и гейзеры Роторуа демонстрируют разнообразие ландшафтов. Хоббитон, декорации из фильмов «Властелин колец», привлекают поклонников Толкина. Маори, коренное население, сохраняет свою уникальную культуру. Адреналиновые развлечения, от банджи-джампинга до рафтинга, доступны по всей стране.",

    "Кения предлагает классическое африканское сафари. Масаи-Мара, Амбосели и Цаво — национальные парки, где можно увидеть «большую пятерку» африканских животных в их естественной среде. Ежегодная миграция антилоп гну — одно из самых впечатляющих природных зрелищ. Пляжи Момбасы с коралловыми рифами идеальны для дайвинга и снорклинга. Племена масаи и самбуру сохраняют традиционный образ жизни.",

    "Вьетнам сочетает богатую историю и динамичное настоящее. Бухта Халонг с ее карстовыми островами — природное чудо. Хойан очаровывает древними улочками и бумажными фонариками. Дельта Меконга предлагает возможность познакомиться с сельской жизнью. Вьетнамская кухня, от фо до свежих спринг-роллов, покоряет своей свежестью. Система туннелей Ку-Чи напоминает о недавней войне.",

    "Антарктида — последний неосвоенный континент, привлекающий самых отважных путешественников. Круизы из Ушуаи позволяют увидеть айсберги, пингвинов и китов. Пересечение пролива Дрейка — настоящее испытание для морских путешественников. Исследовательские станции разных стран ведут научную работу в суровых условиях. Полуночное солнце летом создает сюрреалистичные пейзажи ледяной пустыни."
]

In [9]:

import sys
import subprocess
import importlib

def install_if_needed(package_name, import_name=None):
    """Устанавливает пакет, если он не установлен"""
    if import_name is None:
        import_name = package_name

    try:
        importlib.import_module(import_name)
        print(f"✓ {package_name} уже установлен")
        return True
    except ImportError:
        print(f"Установка {package_name}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"✓ {package_name} успешно установлен")
            return True
        except subprocess.CalledProcessError:
            print(f"✗ Не удалось установить {package_name}")
            return False

install_if_needed("scikit-learn")
install_if_needed("nltk")


import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


exercise_documents = [
    "Машинное обучение — это раздел искусственного интеллекта, изучающий алгоритмы, способные обучаться на данных.",
    "Глубокое обучение является подразделом машинного обучения, использующим нейронные сети с множеством слоев.",
    "Нейронные сети — это вычислительные системы, вдохновленные биологическими нейронными сетями мозга животных.",
    "Обработка естественного языка (NLP) — это область искусственного интеллекта, изучающая взаимодействие компьютеров и человеческого языка.",
    "Компьютерное зрение — это научная дисциплина, изучающая методы извлечения информации из изображений и видео."
]


import nltk

try:
    nltk.download('stopwords', quiet=True)
    print("✓ Ресурс 'stopwords' успешно загружен")
except Exception as e:
    print(f"✗ Ошибка загрузки 'stopwords': {e}")

    stop_words = {'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она',
                  'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее',
                  'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда',
                  'даже', 'ну', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас',
                  'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может',
                  'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была',
                  'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда',
                  'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом',
                  'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех',
                  'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над',
                  'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве',
                  'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше',
                  'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'}
else:

    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('russian'))

print(f"\nИспользуется {len(stop_words)} стоп-слов")


print("\n" + "="*50)
print("ПРЕДОБРАБОТКА ТЕКСТА")
print("="*50)

processed_docs = []
for i in range(len(exercise_documents)):

    doc_lower = exercise_documents[i].lower()


    doc_clean = re.sub(r'[^\w\s]', '', doc_lower)


    words = doc_clean.split()
    filtered_words = [word for word in words if word not in stop_words]
    processed_doc = " ".join(filtered_words)
    processed_docs.append(processed_doc)

    print(f"\nДокумент {i+1}:")
    print(f"  Оригинал: {exercise_documents[i][:50]}...")
    print(f"  После обработки: {processed_doc[:50]}...")


exercise_documents = processed_docs

print("\n" + "="*50)
print("АНАЛИЗ BAG OF WORDS")
print("="*50)


count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(exercise_documents)
bow_feature_names = count_vectorizer.get_feature_names_out()

print(f"\nСловарь содержит {len(bow_feature_names)} уникальных слов")
print("Слова в словаре:", bow_feature_names[:10], "..." if len(bow_feature_names) > 10 else "")

print("\nТоп-3 слова по Bag of Words:\n")

for doc_idx in range(bow_matrix.shape[0]):
    bow_values = bow_matrix[doc_idx].toarray()[0]
    top_indices = np.argsort(bow_values)[-3:][::-1]

    top_words = [(bow_feature_names[i], int(bow_values[i]))
                 for i in top_indices if bow_values[i] > 0]


    if len(top_words) < 3:
        all_words = [(bow_feature_names[i], int(bow_values[i]))
                     for i in range(len(bow_values)) if bow_values[i] > 0]
        all_words_sorted = sorted(all_words, key=lambda x: x[1], reverse=True)
        top_words = all_words_sorted[:3]

    print(f"Документ {doc_idx + 1}: {top_words}")

print("\n" + "="*50)
print("АНАЛИЗ TF-IDF")
print("="*50)


tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(exercise_documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

def tfidf_top3_words(tfidf_matrix, feature_names, top_n=3):
    print(f"\nТоп-{top_n} слова по TF-IDF:\n")

    for doc_idx in range(tfidf_matrix.shape[0]):
        tfidf_values = tfidf_matrix[doc_idx].toarray()[0]


        non_zero_indices = np.where(tfidf_values > 0)[0]

        if len(non_zero_indices) > 0:

            sorted_indices = non_zero_indices[np.argsort(tfidf_values[non_zero_indices])[::-1]]
            top_indices = sorted_indices[:top_n]

            top_words = [(feature_names[i], round(float(tfidf_values[i]), 3))
                         for i in top_indices]
        else:
            top_words = []


        if len(top_words) < top_n and len(non_zero_indices) > 0:
            top_words = [(feature_names[i], round(float(tfidf_values[i]), 3))
                         for i in non_zero_indices[:top_n]]

        print(f"Документ {doc_idx + 1}: {top_words}")

tfidf_top3_words(tfidf_matrix, tfidf_feature_names)


print("\n" + "="*50)
print("СРАВНЕНИЕ BOW И TF-IDF")
print("="*50)

for doc_idx in range(len(exercise_documents)):
    print(f"\nДокумент {doc_idx + 1}:")


    bow_values = bow_matrix[doc_idx].toarray()[0]
    bow_top_indices = np.argsort(bow_values)[-3:][::-1]
    bow_top_words = [(bow_feature_names[i], int(bow_values[i]))
                     for i in bow_top_indices if bow_values[i] > 0]


    tfidf_values = tfidf_matrix[doc_idx].toarray()[0]
    tfidf_top_indices = np.argsort(tfidf_values)[-3:][::-1]
    tfidf_top_words = [(tfidf_feature_names[i], round(float(tfidf_values[i]), 3))
                       for i in tfidf_top_indices if tfidf_values[i] > 0]

    print(f"  BoW:    {bow_top_words}")
    print(f"  TF-IDF: {tfidf_top_words}")


    bow_words_set = set([word for word, _ in bow_top_words])
    tfidf_words_set = set([word for word, _ in tfidf_top_words])
    common_words = bow_words_set.intersection(tfidf_words_set)

    if common_words:
        print(f"  Общие слова в топ-3: {', '.join(common_words)}")


print("\n" + "="*50)
print("СВОДНАЯ ИНФОРМАЦИЯ")
print("="*50)

print(f"Всего документов: {len(exercise_documents)}")
print(f"Всего уникальных слов в словаре: {len(bow_feature_names)}")


print("\nХарактеристики документов:")
print("-" * 60)
print(f"{'Док.':<5} {'Слов (ориг.)':<15} {'Слов (очищ.)':<15} {'Уникальных слов':<15}")
print("-" * 60)

for i, doc in enumerate(exercise_documents):
    original_words = len(re.sub(r'[^\w\s]', '', exercise_documents[i].lower()).split())
    cleaned_words = len(doc.split()) if doc else 0
    unique_words = len(set(doc.split())) if doc else 0

    print(f"{i+1:<5} {original_words:<15} {cleaned_words:<15} {unique_words:<15}")


Установка scikit-learn...
✓ scikit-learn успешно установлен
✓ nltk уже установлен
✓ Ресурс 'stopwords' успешно загружен

Используется 151 стоп-слов

ПРЕДОБРАБОТКА ТЕКСТА

Документ 1:
  Оригинал: Машинное обучение — это раздел искусственного инте...
  После обработки: машинное обучение это раздел искусственного интелл...

Документ 2:
  Оригинал: Глубокое обучение является подразделом машинного о...
  После обработки: глубокое обучение является подразделом машинного о...

Документ 3:
  Оригинал: Нейронные сети — это вычислительные системы, вдохн...
  После обработки: нейронные сети это вычислительные системы вдохновл...

Документ 4:
  Оригинал: Обработка естественного языка (NLP) — это область ...
  После обработки: обработка естественного языка nlp это область иску...

Документ 5:
  Оригинал: Компьютерное зрение — это научная дисциплина, изуч...
  После обработки: компьютерное зрение это научная дисциплина изучающ...

АНАЛИЗ BAG OF WORDS

Словарь содержит 47 уникальных слов
Слова в слов